Данный проект разделен на 2 части:
1. Необходимо написать 2 класса IntegerField и CharField


- IntegerField - принимает только целочисленные значения между заданным диапазоном (min_value и max_value)


- CharField - принимает только строки между заданным диапазоном длины (min_length и max_length)


2. Необходимо создать класс BaseValidator от которого, могут наследоваться IntegerField и CharField

In [183]:
import numbers
import unittest

### Part 1

In [184]:
class IntegerField():
    def __init__(self, min_value=None, max_value=None):
        self._min_value=min_value
        self._max_value=max_value
        
    def __set_name__(self, owner_class, property_name):
        self.prop_name = property_name
    
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name)
        
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise TypeError(f'{self.prop_name} must be integer')
        if self._min_value is not None and value < self._min_value:
            raise ValueError(f'{self.prop_name} must be great {self._min_value}')
        if self._max_value is not None and value > self._max_value:
            raise ValueError(f'{self.prop_name} must be least {self._max_value}')
        instance.__dict__[self.prop_name] = value

class CharField():
    def __init__(self, min_length=0, max_length=None):
        min_length = min_length or 0
        min_length = self._validate(min_length, 'min_length')
        max_length = self._validate(max_length, 'max_length')
        if max_length is not None and min_length is not None and max_length < min_length:
            raise ValueError(f'Max lenght must be >= min length')
        
        self._min_length=min_length
        self._max_length=max_length
        
    def __set_name__(self, owner_class, property_name):
        self.prop_name = property_name
    
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name)
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError(f'{self.prop_name} must be string')
        if self._min_length is not None and len(value) < self._min_length:
            raise ValueError(f'Length {self.prop_name} must be great {self._min_length}')
        if self._max_length is not None and len(value) > self._max_length:
            raise ValueError(f'Length {self.prop_name} must be least {self._max_length}')
        instance.__dict__[self.prop_name] = value
    
    def _validate(self, length, name):
        if length is not None:
            if not isinstance(length, numbers.Integral):
                raise(TypeError(f'{name} must be integral'))
                
            if length < 0:
                return 0
        
        return length

In [185]:
class Person:
    name = CharField(1, 50)
    age = IntegerField(0, 200)
    
    def __init__(self, name, age):
        self.name = name
        self.age = age

#### Tests for part 1

In [186]:
def run_test(cls_test):
    suite = unittest.TestLoader().loadTestsFromTestCase(cls_test)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [189]:
class TestsFields(unittest.TestCase):
    def setUp(self):
        self._name = 'Tom'
        self._age = 20
    
    def test_create(self):
        p = Person('Tom', 20)
        
        self.assertEqual(p.name, self._name)
        self.assertEqual(p.age, self._age)
    
    def test_change_age_name(self):
        p = Person(self._name, self._age)
        new_name = 'Jerry'
        new_age = 15
        
        p.name = new_name
        self.assertEqual(new_name, p.name)
        p.age = new_age
        self.assertEqual(new_age, p.age)
    
    def test_invalid_type(self):
        with self.assertRaises(TypeError):
            Person(name=['T', 'o', 'm'], age=self._age)
        
        with self.assertRaises(TypeError):
            Person(name=self._name, age='20')
        
        with self.assertRaises(TypeError):
            p = Person(name=self._name, age=self._age)
            p.age = '20'
            
        with self.assertRaises(TypeError):
            p = Person(name=self._name, age=self._age)
            p.name = 356
    
    def test_invalid_value(self):
        with self.assertRaises(ValueError):
            Person(name='', age=self._age)
        
        with self.assertRaises(ValueError):
            Person(name=self._name, age=500)
        
        with self.assertRaises(ValueError):
            p = Person(name=self._name, age=self._age)
            p.age = 500
            
        with self.assertRaises(ValueError):
            p = Person(name=self._name, age=self._age)
            p.name = ''

In [190]:
run_test(TestsFields)

test_change_age_name (__main__.TestsFields) ... ok
test_create (__main__.TestsFields) ... ok
test_invalid_type (__main__.TestsFields) ... ok
test_invalid_value (__main__.TestsFields) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.005s

OK


### Part 2

In [191]:
class BaseValidator:
    def __init__(self, min_value=None, max_value=None):
        if max_value is not None and min_value is not None and max_value < min_value:
            raise ValueError(f'Max value must be >= min value')
        
        self._min = min_value
        self._max = max_value
    
    def __set_name__(self, owner_class, property_name):
        self.prop_name = property_name
    
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.prop_name)
    
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self.prop_name] = value
    
    def validate(self, value):
        pass

In [192]:
class IntegerField(BaseValidator):
    def validate(self, value):
        if not isinstance(value, numbers.Integral):
            raise TypeError(f'{self.prop_name} must be an integer.')
        if self._min is not None and value < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min}.')
        if self._max is not None and value > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max}')

class CharField(BaseValidator):
    def __init__(self, min_, max_):
        min_ = max(min_ or 0, 0)
        super().__init__(min_, max_)
        
    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'{self.prop_name} must be a string.')
        if self._min is not None and len(value) < self._min:
            raise ValueError(f'{self.prop_name} must be >= {self._min} chars.')
        if self._max is not None and len(value) > self._max:
            raise ValueError(f'{self.prop_name} must be <= {self._max} chars')

In [193]:
class Person:
    name = CharField(1, 50)
    age = IntegerField(0, 200)
    
    def __init__(self, name, age):
        self.name = name
        self.age = age

#### Tests for part 2

In [194]:
class TestsValidator(unittest.TestCase):
    def setUp(self):
        self._name = 'Tom'
        self._age = 20
    
    def test_create(self):
        p = Person('Tom', 20)
        
        self.assertEqual(p.name, self._name)
        self.assertEqual(p.age, self._age)
    
    def test_change_age_name(self):
        p = Person(self._name, self._age)
        new_name = 'Jerry'
        new_age = 15
        
        p.name = new_name
        self.assertEqual(new_name, p.name)
        p.age = new_age
        self.assertEqual(new_age, p.age)
    
    def test_invalid_type(self):
        with self.assertRaises(TypeError):
            Person(name=['T', 'o', 'm'], age=self._age)
        
        with self.assertRaises(TypeError):
            Person(name=self._name, age='20')
        
        with self.assertRaises(TypeError):
            p = Person(name=self._name, age=self._age)
            p.age = '20'
            
        with self.assertRaises(TypeError):
            p = Person(name=self._name, age=self._age)
            p.name = 356
    
    def test_invalid_value(self):
        with self.assertRaises(ValueError):
            Person(name='', age=self._age)
        
        with self.assertRaises(ValueError):
            Person(name=self._name, age=500)
        
        with self.assertRaises(ValueError):
            p = Person(name=self._name, age=self._age)
            p.age = 500
            
        with self.assertRaises(ValueError):
            p = Person(name=self._name, age=self._age)
            p.name = ''

In [195]:
run_test(TestsValidator)

test_change_age_name (__main__.TestsValidator) ... ok
test_create (__main__.TestsValidator) ... ok
test_invalid_type (__main__.TestsValidator) ... ok
test_invalid_value (__main__.TestsValidator) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK
